In [1]:
'''save data to lmdb database'''
import lmdb
import pandas as pd
import torch 
import rdkit
from  tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler

import numpy as np
import torch
import random
from rdkit import Chem
from scipy.spatial import distance_matrix
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import pickle
import dgl
import dgl.data

/home/caoduanhua/anaconda3/envs/prolif/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# key = '/home/caoduanhua/score_function/data/general_refineset/pdbbind-shape_decoy_align_pocket-5/5vsb_ligand_align_3'
# # try:
# try:
#     with open(key, 'rb') as f:
#         m1,m2= pickle.load(f)
#     # f.close()
#     # txn.put(key.encode(), pickle.dumps(m1,m2), db=mol_pocket_inter_idx_type_db)
# except:
#     with open(key, 'rb') as f:
#         m1,m2,atompairs,iter_types= pickle.load(f)
#         # f.close()
#         # txn.put(key.encode(), pickle.dumps(m1,m2,atompairs,iter_types), db=mol_pocket_inter_idx_type_db)
# # except:
# #     print('file: {} is not a valid file!'.format(key))

In [4]:
import tqdm
# 创建数据库文件
env = lmdb.open(f'/home/caoduanhua/score_function/data/lmdbs/pose_challenge_cross_10', map_size=int(1e12), max_dbs=1)
# 创建对应的数据库
# mol_pocket_inter_idx_type = env.open_db('mol_pocket_interIdx_type'.encode())
mol_pocket_inter_idx_type_db = env.open_db('data'.encode())
# 把数据写入到LMDB中

with open ('./train_keys.pkl', 'rb') as fp:
    train_keys = pickle.load(fp)
with open ('./val_keys.pkl', 'rb') as fp:
    val_keys = pickle.load(fp)
with open ('./test_keys.pkl', 'rb') as fp:
    test_keys = pickle.load(fp)

keys = test_keys + val_keys + train_keys
print('len of keys',len(keys))
all_keys = len(keys)
pbar = tqdm.tqdm(keys)
with env.begin(write=True) as txn:
    for i ,key in enumerate(pbar):
        try:
            try:
                with open(key, 'rb') as f:
                    m1,m2= pickle.load(f)
                # f.close()
                txn.put(key.encode(), pickle.dumps((m1,m2)), db=mol_pocket_inter_idx_type_db)
            except:
                with open(key, 'rb') as f:
                    m1,m2,atompairs,iter_types= pickle.load(f)
                # f.close()
                txn.put(key.encode(), pickle.dumps((m1,m2,atompairs,iter_types)), db=mol_pocket_inter_idx_type_db)
        except:
            print('file: {} is not a valid file!'.format(key))
            continue
        pbar.update()
        pbar.set_description(f'processing {i+1}/{all_keys}')
env.close()



len of keys 449125


processing 329429/449125:  74%|███████▍  | 333305/449125 [25:16<08:11, 235.70it/s] 

file: /home/caoduanhua/score_function/data/D-PDBbind_PDBscreen/PDB_bind_cross_decoy_pocket/4o43_decoy_2zdl_0 is not a valid file!


processing 449125/449125: 100%|██████████| 449125/449125 [35:05<00:00, 213.30it/s]


In [23]:
env = lmdb.open(f'/home/caoduanhua/score_function/data/lmdbs/pose_challenge_cross', map_size=int(1e12), max_dbs=1, readonly=True)

graph_db = env.open_db('data'.encode()) # graph data base
key = keys[0] + 'bnm'
# try:

with env.begin(write=False) as txn:
    m1,m2, atompairs,iter_types,_= pickle.loads(txn.get(key.encode(), db=graph_db))
# except:
    # print('not in database plz to processcing')
    # pass

TypeError: a bytes-like object is required, not 'NoneType'

['Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic',
 'Hydrophobic']

In [ ]:
try:
    m1,m2, atompairs,iter_types= pickle.loads(txn.get(key.encode(), db=graph_db))
    # pass
except ValueError:
    m1,m2= pickle.loads(txn.get(key.encode(), db=graph_db))
    # pass
except TypeError:
    pass

finally:


